In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [62]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
class QLearning(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim):
        super(QLearning, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.dropout = nn.Dropout(0.5)
        # Two actions: 0 or 1
        self.fc = nn.Linear(hidden_dim * 2, 2)  

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        q_values = self.fc(out)
        return q_values

def update_q_values(model, optimizer, states, actions, rewards, next_states, gamma=0.99):
    model.eval()
    with torch.no_grad():
        q_values_next = model(next_states).max(1)[0]  # Get max Q value for next states
    model.train()
    q_values = model(states).gather(1, actions.unsqueeze(-1)).squeeze(-1)
    
    # Q-Learning update target
    target = rewards + gamma * q_values_next
    loss = nn.functional.mse_loss(q_values, target)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [63]:
def train(model, device, train_loader, optimizer, epoch, gamma=0.99, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Predict Q-values for current states (data)
        current_q_values = model(data)

        # Simulate taking actions (classifying) and receiving rewards
        # In a real RL scenario, actions are taken based on a policy. Here, we simplify.
        _, predicted_actions = torch.max(current_q_values, 1)
        rewards = (predicted_actions == target).float()  # Reward is 1 for correct classification, 0 otherwise

        # Simulate next states (in practice, you might use a different strategy)
        # For simplicity, let's just roll data to simulate "next states"
        next_data = torch.roll(data, -1, 0)
        with torch.no_grad():
            future_q_values = model(next_data).max(1)[0]  # Use max Q-value for simplicity
        
        # Compute target Q-values
        target_q_values = rewards + gamma * future_q_values

        # Update model based on the difference between current Q-values and target Q-values
        # Assuming binary classification, gather Q-values for the taken actions
        action_indices = target.view(-1, 1).long()
        gathered_q_values = current_q_values.gather(1, action_indices).squeeze()

        loss = nn.functional.mse_loss(gathered_q_values, target_q_values.detach())
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')


In [64]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Validation Encoded text: tensor([[ 2, 52,  0,  0,  0,  0,  0,  0,  0,  0],
        [55,  6, 23, 31, 58, 47,  0,  0,  0,  0],
        [62, 71, 12,  0,  0,  0,  0,  0,  0,  0],
        [26, 42,  8, 23, 30,  0,  0,  0,  0,  0]])
Validation Label: tensor([0, 1, 1, 0])
Validation Encoded text: tensor([[13, 12,  9,  0,  0,  0,  0,  0,  0,  0],
        [10, 20,  2, 11,  0,  0,  0,  0,  0,  0],
        [10,  1, 14,  5, 25, 17,  0,  0,  0,  0],
        [22, 23,  4, 15,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[11, 16,  7, 28, 31,  0,  0,  0,  0,  0],
        [ 2, 17,  4,  9,  2, 27,  1,  0,  0,  0],
        [18, 15, 24,  0,  0,  0,  0,  0,  0,  0],
        [12,  3, 10,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [65]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QLearning(vocabulary_size, embedding_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.741600

Validation set: Average loss: 0.1440, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.928177

Validation set: Average loss: 0.1322, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.885316

Validation set: Average loss: 0.1290, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.588004

Validation set: Average loss: 0.1301, Accuracy: 7/8 (88%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.541033

Validation set: Average loss: 0.1308, Accuracy: 7/8 (88%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.216209

Validation set: Average loss: 0.1331, Accuracy: 7/8 (88%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.515278

Validation set: Average loss: 0.1295, Accuracy: 7/8 (88%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.788985

Validation set: Average loss: 0.1203, Accuracy: 7/8 (88%)

Train Epoch: 9 [0/24 (0%)]	Loss: 2.344689

Validation set: Average loss: 0.1177, Accuracy: 7/8 (88%)

Train Epoch: 10 [0/24 (0%)]	Loss: 1.533728

Validation set: Average loss: 0.1240, 